In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 25 10:49:25 2025

@author: mbozh
"""

import pandas as pd
import networkx as nx 
import numpy as np
import random 
import math
import matplotlib.pyplot as plt

def create_network(df):
   """
   Converts a DataFrame with source nodes, target nodes and edge weights
   into a directed NetworkX graph.

   Parameters:
   ----------
   df : pandas.DataFrame
       A DataFrame with the following columns:
       - 'source': Source node (str or int)
       - 'target': Target node (str or int)
       - 'weight': Weight of the directed edge (int or float)

   Returns:
   -------
   G : networkx.DiGraph
       A directed graph where each edge from 'source' to 'target' has an associated weight.
   """
   
   G = nx.DiGraph() # Initialize a directed graph

   # Iterate through each row to add edges with weights
   for _, row in df.iterrows():
       source = row['source']
       target = row['target']
       weight = row['weight']
       G.add_edge(source, target, weight=weight)

   return G

'''
#Generate random dataframe to check the code:
df = pd.DataFrame({
    'source': np.random.randint(0, 500, size=1000),
    'target': np.random.randint(0, 500, size=1000),
    'weight': np.random.rand(1000)
})

#Built in networkx
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight', create_using=nx.DiGraph(), edge_key=None)

#function check:
G = create_network(df) #quick enough
'''

file_path = './Weighted_network_data_3.csv' 
df = pd.read_csv(file_path)
G = create_network(df)

#network analysis
disconnected_edges = []
for component in nx.weakly_connected_components(G):  # for directed graph G
    subgraph = G.subgraph(component)
    if subgraph.number_of_nodes() == 2 and subgraph.number_of_edges() == 1:
        disconnected_edges.extend(subgraph.edges())

#len(disconnected_edges) #252

largest_cc_nodes = max(nx.weakly_connected_components(G),key=len) #4765 nodes
largest_cc_subgraph = G.subgraph(largest_cc_nodes).copy()

r'''
#transform weights to negative log scale:
G_log = nx.DiGraph()  

for u, v, data in largest_cc_subgraph.edges(data=True):
    w = data.get('weight', 1)
    if w <= 0:
        raise ValueError(f"Edge ({u}, {v}) has non-positive weight: {w}")
    G_log.add_edge(u, v, weight=-math.log(w))

paths = dict(nx.all_pairs_dijkstra_path(G_log))
path_lengths = dict(nx.all_pairs_dijkstra_path_length(G_log))

import json

# Use the same 'records' list from before
records = []
for src in G_log.nodes():
    for dst in G_log.nodes():
        if src != dst and dst in paths[src]:
            records.append({
                "source": src,
                "target": dst,
                "path": paths[src][dst],
                "neg_log_weight": path_lengths[src][dst]
            })

# Save to JSON file
with open("shortest_paths_neg_log.json", "w") as f:
    json.dump(records, f, indent=2)
##### Collect the paths: 

'''
zero_out_degree_nodes = [node for node in largest_cc_subgraph.nodes if largest_cc_subgraph.out_degree(node) == 0]
#1584
in_degrees_list = []
for node in zero_out_degree_nodes:
    in_degrees_list.append(len(list(largest_cc_subgraph.predecessors(node))))

#plt.hist(in_degrees_list,bins=max(in_degrees_list))
#plt.xlabel('In-degree')
#plt.ylabel('Frequency')

largest_cc_subgraph.remove_edges_from(nx.selfloop_edges(largest_cc_subgraph))
    
def collect_paths(G, source, target, num_paths=1000, max_steps=100):
    """
    Perform random walks on a directed weighted graph from source to target.

    Parameters:
    ----------
    G : networkx.DiGraph
        A directed graph with edge weights used as transition probabilities.
    source : node
        Starting node for the random walk.
    target : node
        Ending node to stop the walk.
    num_paths : int
        Number of random walk paths to collect (default = 1000).
    max_steps : int
        Maximum steps per walk to avoid infinite loops (default = 100).

    Returns:
    -------
    paths : list of lists
        A list containing up to `num_paths` paths (each a list of nodes) from source to target.
    """
    paths = []

    for _ in range(num_paths):
        path = [source]
        current = source
        steps = 0

        while current != target and steps < max_steps:
            neighbors = list(G.successors(current))
            if not neighbors:
                break  # dead end
            probabilities = [G[current][nbr].get('weight', 1.0) for nbr in neighbors]
            #print(probabilities)
            current = random.choices(neighbors, weights=probabilities, k=1)[0]
            path.append(current)
            #print(path)
            steps += 1
            
            # If we reach a stop/sink node (zero out degree),
            # stop current path realization
            if largest_cc_subgraph.out_degree(current) == 0:
                continue

        #if path[-1] == target:
        paths.append(path)

    return paths





In [3]:
df

,Unnamed: 0,target,source,number_people,weight
0,0,11-Janaay-1,Bardheere,9,0.473684
1,1,11-Janaay-1,Bardheere,3,1.000000
2,2,11-Janaay-1,Diinsoor,8,0.014035
3,3,11-Janaay-1,Maanyow,4,0.014388
4,4,11-Janaay-1,Qansaxdheere,4,0.031746
...,...,...,...,...,...
9578,9578,iidko,Kurtunwaarey,5,0.050505
9579,9579,kaah,Warabaale,28,1.000000
9580,9580,kismaayo,Halgan,25,1.000000
9581,9581,warjanaay,Mukoy dhere,26,1.000000


In [81]:
paths = dict()
from tqdm import tqdm
for source in list(G.nodes())[1:2]:
    for target in list(G.nodes())[45:46]:
        if target!=source:
            paths[(source,target)] =  collect_paths(G, source, target, num_paths=1000, max_steps=10**7)
        

source = "Qod-Qod"
target = "Baraka"
paths = collect_paths(largest_cc_subgraph, source, target, num_paths=10000, max_steps=10**6)


In [85]:
paths

[['Qod-Qod', 'Kanaanah', 'Lebiley'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Kanaanah', 'Tuni Camp'],
 ['Qod-Qod', 'Kanaanah', 'Lebiley'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod',
  'Kanaanah',
  'Horseed',
  'Bakaley',
  'Waradoy',
  'Buur Madow',
  'Hillaac',
  'Madina',
  'Howlwadag2'],
 ['Qod-Qod', 'Kanaanah', 'Raxole Camp'],
 ['Qod-Qod', 'Kanaanah', 'Horseed', 'Sacad'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Kanaanah', 'Horseed', 'Yeed', 'Shilow'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Kanaanah', 'Tuni Camp'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Kananax Camp'],
 ['Qod-Qod', 'Kanaanah', 'Raxole Camp'],
 ['Qod-Qod', 'Qot Qot Camp'],
 ['Qod-Qod', 'Kanaanah', 'Horseed', 'Baladulamiin'],
 ['Qod-Q

In [38]:
def all_pairs_max_weight_paths(G, weight='weight', max_hops=10):
    """
    Find the maximum-weight simple path between all node pairs in a cyclic directed graph.

    Parameters:
    -----------
    G : networkx.DiGraph
        Directed graph (can have cycles).
    weight : str
        Edge attribute used for weight.
    max_hops : int
        Max length of paths to explore (cutoff for all_simple_paths).

    Returns:
    --------
    result : dict
        Mapping (source, target) -> {'path': [...], 'weight': ...}
    """
    result = {}
    nodes = list(G.nodes())

    for source in nodes:
        for target in nodes:
            if source == target:
                continue
            try:
                paths = nx.all_simple_paths(G, source=source, target=target, cutoff=max_hops)
                max_weight = float('-inf')
                best_path = None
                for path in paths:
                    total_weight = sum(G[u][v].get(weight, 1.0) for u, v in zip(path[:-1], path[1:]))
                    if total_weight > max_weight:
                        max_weight = total_weight
                        best_path = path
                if best_path:
                    result[(source, target)] = {'path': best_path, 'weight': max_weight}
            except nx.NetworkXNoPath:
                continue  # skip unreachable pairs

    return result
